In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import factory, simulate, scipy, re, os, GPy
import pandas as pd

In [ ]:
reload(factory)
reload(simulate)

In [ ]:
from GPy.core import Mapping, Param
from paramz.transformations import Logexp
import paramz.transformations

In [ ]:
sim = simulate.Simulation(batchVariance=.1,repVariance=0.05,nobs=10)
sim

In [ ]:
sim.generateSamples()

In [ ]:
for ds in sim.datasets:
    sim.plotSamples(ds)

In [ ]:
for ds in sim.datasets:

    y0, y1, y2, y3 = sim.splitSample(ds['sample'])

    f = (sim.f-ds['mean'])/ds['std']

    gp, trained = sim.m3.batchTrain(y3,10)
    
    print sum([gp.log_likelihood()<t.log_likelihood() for t in trained]) <= 1
    
    plt.figure()
    plt.hist([t.log_likelihood() for t in trained])
    
    #gp.plot_f(fixed_inputs=[(1,-1), (2,-1)])
    
    plt.figure()
    
    mu, cov = gp.predict_noiseless(sim.xpred,kern=sim.m3.predictionKernel(gp.kern))
    mu = mu[:,0]
    std = np.sqrt(cov[:,0])

    plt.plot(sim.xpred[:,0], mu)
    plt.fill_between(sim.xpred[:,0], mu-2*std,mu+2*std, alpha=.4)

In [ ]:
sim.xpred

In [ ]:
sum([gp.log_likelihood()<t.log_likelihood() for t in trained]) <= 1

In [ ]:
plt.hist([t.log_likelihood() for t in trained])

In [ ]:
gp.plot_f(fixed_inputs=[(1,-1), (2,-1)])

In [ ]:
g = gompertz.Gompertz(1,.3,1)
g

In [ ]:
x = np.linspace(0,2)[:,None]
y = g.f(x) + scipy.stats.norm.rvs(scale=.05,size=50)[:,None]


plt.plot(g.f(x))
plt.plot(y);

In [ ]:
g.A.gradient

In [ ]:
g = gompertz.Gompertz(1,.3,1)

temp = GPy.models.GPRegression(x,y, mean_function=g.copy())
temp.randomize()
temp.optimize()
temp

In [ ]:
plt.plot(g.f(x))
plt.plot(temp.gompertz.f(x))
plt.plot(y)

In [ ]:
temp.gompertz.param_array

In [ ]:
import gompertz
reload(gompertz)

In [ ]:
import gompertz
reload(gompertz)

plt.plot(g.f(x))
plt.plot(y)

params = []

for i in range(20):
    temp = GPy.models.GPRegression(x,y, mean_function=gompertz.Gompertz(1,1,1))
    temp.randomize()
    temp.optimize()
    
    plt.plot(temp.gompertz.f(x))
    
    params.append(temp.gompertz.param_array)

In [ ]:
plt.boxplot(np.array(params));
plt.plot(range(1,4), g.param_array)

In [ ]:
dist = np.sum((np.array(params)-g.param_array)**2,1)

params[np.where(dist==dist.min())[0][0]]
# np.where(dist==dist.min())[0]

In [ ]:
g

In [ ]:
temp2 = GPy.models.GPRegression(sim.x0, sim.m0.reshape(sim.splitSample(sim.datasets[0])[0]))

In [ ]:
temp.plot_f()

In [ ]:
temp2.plot_f()

In [ ]:
g

In [ ]:
p = GPy.priors.Gamma(5,1000)

In [ ]:
p.plot()

In [ ]:
def addPriors(m):
    m.kern.level0.variance.set_prior(GPy.priors.Gamma(5,1000))
    m.kern.level1.variance.set_prior(GPy.priors.LogGaussian(5,1000))
    
    return m

In [ ]:
gp = GPy.models.GPRegression(sim.x0, sim.m0.reshape(sim.splitSample(sim.datasets[0])[1]), mean_function=gompertz.Gompertz(1,1,1))
gp.randomize()
gp

In [ ]:
gp.optimize()
gp.plot_f()
plt.plot(sim.x0[:15,0],sim.f[:15,])

In [ ]:
params = []

plt.plot(sim.f[:15],c='k',lw=3)

for i in range(20):
    temp = GPy.models.GPRegression(sim.x0, sim.m0.reshape(sim.splitSample(sim.datasets[0])[0]), mean_function=gompertz.Gompertz(1,1,1))
    temp.randomize()
    temp.optimize()
    
    plt.plot(temp.gompertz.f(sim.x0[:15]))
    
    params.append(temp.gompertz.param_array)

In [ ]:
plt.boxplot(np.array(params));
plt.plot(range(1,4), [2,.4,1])

In [ ]:
gp, trained = sim.m3.batchTrain(sim.splitSample(sim.datasets[0])[1], 5, callback=addPriors)

In [ ]:
gp

In [ ]:
gp.kern.level0.variance = 1e-10
gp.kern.level1.variance = .04

In [ ]:
plt.imshow(gp.kern.K(gp.X))

In [ ]:
def extractParam(m, *args):
    if len(args) == 0:
        return m
    else:
        if args[0] in m.__dict__:
            return extractParam(m.__dict__[args[0]], *args[1:])
        else:
            return np.nan

In [ ]:
p = extractParam(gp, *'kern.fdsafdsa.variance'.split("."))

In [ ]:
type(p) == float

In [ ]:
p.values

In [ ]:
p

In [ ]:
[t.log_likelihood() for t in trained]

In [ ]:
gp.plot_f(fixed_inputs=[(1,-1), (2,-1)])

In [ ]:
[t.plot_f(fixed_inputs=[(1,-1), (2,-1)]) for t in trained]

In [ ]:
intervals = {}
accuracy = {}

for ds in sim.datasets:

    plt.figure(figsize=(10,10))

    models = {}
    for i,y in enumerate(sim.splitSample(ds)):

        ax = plt.subplot2grid((4,5),(i,0))
        plt.plot(y);

        for j, m in enumerate([sim.m0, sim.m1, sim.m2, sim.m3]):
            ax = plt.subplot2grid((4,5),(i,1+j))

            gp,_ = m.batchTrain(y)

            models[(i,j)] = gp

            mu, cov = gp.predict_noiseless(sim.xpred,kern=m.predictionKernel(gp.kern))
            mu = mu[:,0]
            std = np.sqrt(cov[:,0])

            plt.plot(sim.xpred[:,0], mu)
            plt.fill_between(sim.xpred[:,0], mu-2*std,mu+2*std, alpha=.4)
            
            if not (i,j) in intervals:
                intervals[(i,j)] = 0
                accuracy[(i,j)] = []
                
            if all((sim.f[:sim.nobs] > mu-thresh*std) & (sim.f[:sim.nobs] < mu+thresh*std)):
                intervals[(i,j)] += 1
            accuracy[(i,j)].append(1.*sum((sim.f[:sim.nobs] > mu-thresh*std) & (sim.f[:sim.nobs] < mu+thresh*std))/sim.nobs)

            plt.plot(sim.x[:sim.nobs,0], sim.f[:sim.nobs],c='k')


In [ ]:
# pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-([a-zA-Z.0-9]+).csv'
pattern = 'simulation-([0-9.]+-)*([a-zA-Z.0-9]+).csv'

targets = os.listdir("results/")
targets = filter(lambda x: re.match(pattern,x), targets)
targets = map(lambda x: re.match(pattern, x).groups()[-1], targets)

targets = list(set(targets))
targets.sort()
targets

In [ ]:
pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'

for target in targets:

    files = os.listdir("results/")
    files = filter(lambda x: re.match(pattern%target,x), files)
    
    print target, files
    print

In [ ]:
def findFiles(target, pattern='simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'):
    files = os.listdir("results/")
    files = filter(lambda x: re.match(pattern%target,x), files)
    return files

In [ ]:
def loadResults(target, files=None, pattern='simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'):
    
    if files is None:
        files = findFiles(target, pattern)
    
    results = {}
    for f in files:
        nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%target,f).groups()
        results[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)
        
    return results

In [ ]:
def plotResult(res, target, title=False, prefix=None):
    
    if type(res) == dict:
        for k,v in res.iteritems():
            plt.figure(figsize=(10,4))
            plotResult(v.values, target)
            if title:
                plt.suptitle(k,y=1.05)
                
            if not prefix is None:
                plt.savefig("%s-%s-%s.pdf"%(prefix, str(k),target), bbox_inches='tight')
        
        return

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)

        plt.boxplot(res[:,i*4:(i+1)*4],
                       labels=['M%d'%i for i in  range(4)]);

        if 'accuracy' in target:
            plt.plot([1,4], [.95,.95])

        plt.ylim(np.nanmin(res), np.nanmax(res))

    plt.tight_layout()

In [ ]:
def plotBayesFactor(res, title=False):
    
    if type(res) == dict:
        for k,v in res.iteritems():
            plt.figure(figsize=(10,4))
            plotBayesFactor(v.values)
            if title:
                plt.suptitle(k)
        
        return
    
    ylim = (np.inf, -np.inf)

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        
        plt.boxplot(res[:,i*4:(i+1)*4] - res[:,i*4][:,None],
                       labels=['M%d'%i for i in  range(4)]);

#         plt.boxplot(res[:,i*4:(i+1)*4],
#                        labels=['M%d'%i for i in  range(4)]);

        if 'accuracy' in target:
            plt.plot([1,4], [.95,.95])
            
        ylim = (min(ylim[0], np.nanmin(res[:,i*4:(i+1)*4] - res[:,i*4][:,None])), max(ylim[1], np.nanmax(res[:,i*4:(i+1)*4] - res[:,i*4][:,None])))

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.ylim(ylim)

    plt.tight_layout()

In [ ]:
res = loadResults('accuracy')
plt.boxplot(res[res.keys()[0]].values);

In [ ]:
plt.hist(res[res.keys()[0]].values[:,-6])
plt.hist(res[res.keys()[0]].values[:,-5])

In [ ]:
n = 20
np.arange(n+1,dtype=float)/n

In [ ]:
for t in targets:
    res = loadResults(t)
    plotResult(res,t,title=True,prefix='figures/simulation')

In [ ]:
res = loadResults('likelihood')
plotResult(res,title=True)

In [ ]:
plotBayesFactor(res, title=True)

In [ ]:
def plotLikelihoodRatio(res, title=False, prefix=None):
    
    if type(res) == dict:
        for k,v in res.iteritems():
            plt.figure(figsize=(16,4))
            plotLikelihoodRatio(v.values)
            if title:
                plt.suptitle(k,y=1.05)
                
            if not prefix is None:
                plt.savefig("%s-%s-likelihoodRatio.pdf"%(prefix, str(k)), bbox_inches='tight')
        
        return
    
    ylim = (np.inf, -np.inf)

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        
        #diff = 2*(res[:,[i*4+1, i*4+2, i*4+2, i*4+3, i*4+3]] - res[:,[i*4, i*4, i*4+1, i*4+1, i*4+2]])
        diff = 2*(res[:,[i*4+1, i*4+2, i*4+2, i*4+3, i*4+3, i*4+3]] - res[:,[i*4, i*4, i*4+1, i*4+1, i*4+2, i*4]])
        
        plt.boxplot(diff, labels=['M1/M0', 'M2/M0', 'M2/M1', 'M3/M1', 'M3/M2', 'M3/M0']);
        
        plt.plot([.5,5.5], [scipy.stats.chi2.ppf(.95, 2)]*2)
        plt.plot([5.5,6.5], [scipy.stats.chi2.ppf(.95, 4)]*2)
            
        ylim = (min(ylim[0], diff.min()), max(ylim[1], diff.max()))

    for i in range(4):

        plt.subplot(1,4,i+1)
        plt.ylim(ylim)

    plt.tight_layout()

In [ ]:
plotLikelihoodRatio(res, title=True, prefix='figures/simulation-')

In [ ]:
for target in targets:
    res = loadResults(target)
    
    for k,v in res.iteritems():
        plt.figure(figsize=(10,5))
        plotResult(v.values, target)
        
        plt.savefig("figures/simulation-%s-%s.pdf"%(','.join(k), target), bbox_inches='tight')

In [ ]:
pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'

for target in targets:

    files = os.listdir("results/")
    files = filter(lambda x: re.match(pattern%target,x), files)

    results = {}
    for f in files:
        nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%target,f).groups()
        results[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)

    for nobs, nbatch, nrep, sigma, batchVariance, repVariance in results.keys():

        res = results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values

        plt.figure(figsize=(10,5))

        for i in range(4):
            
            ii = i
            if ii == 1:
                ii = 2
            elif ii == 2:
                ii = 1
                
            plt.subplot(1,4,ii+1)
            plt.title("M%d"%ii)
            
            plt.boxplot(res[:,i*4:(i+1)*4],
                           labels=['M%d'%i for i in  range(4)]);
            
            if 'accuracy' in target:
                plt.plot([1,4], [.95,.95])

            plt.ylim(res.min().min(), res.max().max())

        plt.tight_layout()
        plt.savefig("figures/simulation-%s-%s.pdf"%(','.join([nobs, nbatch, nrep, sigma, batchVariance, repVariance]), target), bbox_inches='tight')

In [ ]:
pattern = 'simulation-([0-9]+)-([0-9]+)-([0-9]+)-([0-9.]+)-([0-9.]+)-([0-9.]+)-%s.csv'

target = 'likelihood'

files = os.listdir("results/")
files = filter(lambda x: re.match(pattern%target,x), files)

results = {}
for f in files:
    nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%target,f).groups()
    results[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)

for nobs, nbatch, nrep, sigma, batchVariance, repVariance in results.keys():

    res = results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values

    plt.figure(figsize=(10,5))

    for i in range(4):

        ii = i
        if ii == 1:
            ii = 2
        elif ii == 2:
            ii = 1
            
        plt.subplot(1,4,ii+1)
        plt.title("M%d"%ii)

        plt.boxplot(res[:,i*4:(i+1)*4] - res[:,i*4][:,None],
                       labels=['M%d'%i for i in  range(4)]);

        plt.ylim(-10, 190)

    plt.tight_layout()
    plt.savefig("figures/simulation-%s-bayesFactor.pdf"%(','.join([nobs, nbatch, nrep, sigma, batchVariance, repVariance])), bbox_inches='tight')

In [ ]:
# target='kern.level0.variance'
target='likelihood.variance'

files = os.listdir("results/")
files = filter(lambda x: re.match(pattern%target,x), files)

results = {}
for f in files:
    nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%target,f).groups()
    
    results[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)
    
    
for nobs, nbatch, nrep, sigma, batchVariance, repVariance in results.keys():
    
    acc = results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values
    
    plt.figure(figsize=(10,5))
    
    for i in range(4):
        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        plt.boxplot(results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values[:,i*4:(i+1)*4],
                       labels=['M%d'%i for i in  range(4)]);
        
        #if 'level0.variance' in target:
        #    plt.plot([1,4],[repVariance, repVariance])
        
        plt.ylim(results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].min().min(), results[nobs, nbatch, nrep, sigma, batchVariance, repVariance].max().max())
        
    plt.tight_layout()
    plt.savefig("figures/simulation-%s-%s.pdf"%(','.join([nobs, nbatch, nrep, sigma, batchVariance, repVariance]), target), bbox_inches='tight')

In [ ]:
for nobs, nbatch, nrep, sigma, batchVariance, repVariance in accuracy.keys():
    
    acc = accuracy[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values
    
    plt.figure(figsize=(10,5))
    
    for i in range(4):
        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        plt.boxplot(accuracy[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values[:,i*4:(i+1)*4],
                       labels=['M%d'%i for i in  range(4)]);
        
        plt.ylim(-.1,1.1)
        
    plt.tight_layout()
    plt.savefig("figures/simulation-%s-accuracy.pdf"%','.join([nobs, nbatch, nrep, sigma, batchVariance, repVariance]), bbox_inches='tight')

In [ ]:
files = os.listdir("results/")
files = filter(lambda x: re.match(pattern%'accuracy',x), files)
files

In [ ]:
accuracy = {}
for f in files:
    nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%'accuracy',f).groups()
    
    accuracy[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)

In [ ]:
for nobs, nbatch, nrep, sigma, batchVariance, repVariance in accuracy.keys():
    
    acc = accuracy[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values
    
    plt.figure(figsize=(10,5))
    
    for i in range(4):
        plt.subplot(1,4,i+1)
        plt.title("M%d"%i)
        plt.boxplot(accuracy[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values[:,i*4:(i+1)*4],
                       labels=['M%d'%i for i in  range(4)]);
        
        plt.ylim(-.1,1.1)
        
    plt.tight_layout()
    plt.savefig("figures/simulation-%s-accuracy.pdf"%','.join([nobs, nbatch, nrep, sigma, batchVariance, repVariance]), bbox_inches='tight')

In [ ]:
files = os.listdir("results/")
files = filter(lambda x: re.match(pattern%'likelihood',x), files)
files

In [ ]:
accuracy = {}
for f in files:
    nobs, nbatch, nrep, sigma, batchVariance, repVariance = re.match(pattern%'likelihood',f).groups()
    
    accuracy[(nobs, nbatch, nrep, sigma, batchVariance, repVariance)] = pd.read_csv('results/%s'%f, header=[0,1], index_col=None)

In [ ]:
for nobs, nbatch, nrep, sigma, batchVariance, repVariance in accuracy.keys():
    plt.figure()
    plt.suptitle(str((nobs, nbatch, nrep, sigma, batchVariance, repVariance)), y=1.05)
    
    for i in range(4):
        plt.subplot(1,4,i+1)
        plt.boxplot(accuracy[nobs, nbatch, nrep, sigma, batchVariance, repVariance].values[:,i*4:(i+1)*4]);
        
    plt.tight_layout()

In [ ]:
accuracy.keys()

In [ ]:
for k in intervals.keys():
    intervals[k] = 1.*intervals[k]/len(sim.datasets)

In [ ]:
intervals

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(accuracy)

In [ ]:
m = sim.m3
gp = models[(3,3)]

In [ ]:
alpha = .05
thresh = scipy.stats.norm.ppf((1-alpha/2))
thresh

In [ ]:
mu, cov = gp.predict_noiseless(sim.xpred,kern=m.predictionKernel(gp.kern), full_cov=False)
mu = mu[:,0]
std = np.sqrt(cov[:,0])

In [ ]:
mu, cov = gp.predict_noiseless(sim.xpred,kern=m.predictionKernel(gp.kern), full_cov=True)

In [ ]:
std = np.sqrt(cov.diagonal())

plt.plot(sim.xpred[:,0], mu[:,0])
plt.fill_between(sim.xpred[:,0], mu[:,0]-thresh*std,mu[:,0]+thresh*std, alpha=.4)
plt.plot(sim.x[:sim.nobs,0], sim.f[:sim.nobs], c='k')

In [ ]:
plt.imshow(cov+1e-3*cov.mean()*np.eye(15))

In [ ]:
diff = mu[:,0]-sim.f[:sim.nobs]

In [ ]:
plt.plot(diff)

In [ ]:
np.dot(diff, np.dot(np.linalg.inv(cov+1e-6*cov.mean()*np.eye(15)), diff))

In [ ]:
scipy.stats.chi2.ppf(.95, 15)

In [ ]:
scipy.stats.multivariate_normal.logpdf(sim.f[:sim.nobs], mu[:,0], cov)

In [ ]:
y0, y2 , y1, y3 = sim.splitSample(sim.datasets[0])

In [ ]:
gp = sim.m1.train(y1)

In [ ]:
gp

In [ ]:
gp.plot_f(fixed_inputs=[(1,-1)])
plt.plot(sim.x[:sim.nobs,0], sim.f[:sim.nobs])

In [ ]:
y0.shape, sim.x.shape

In [ ]:
simulate.generateSample(sim.f, sim.sampleCov, sim.sigma, sim.nobs)

In [ ]:
plt.plot(sim.f)